# 在最高得分的12-04-07上将总含量由氮和磷加和得到
#发现5月的浓度值多数不符合正常范围内的条件，若此次模型不合适，建议删除

In [1]:
import pandas as pd

In [2]:
def read_data(file_path,all_data):
    pd_time=pd.read_csv(file_path+'/'+'Z230进口硫酸压力-Raw.csv',header="infer",encoding='gbk',index_col=False)
    pd_time.columns = ["Id", "Time", 'Value']
    pd_time['Time']=pd_time['Time'].str.replace('[','').str.replace(']','')
    pd_time['Time']=pd.to_datetime(pd_time['Time'])
    for file in files:
        data=pd.read_csv(file_path+'/'+file,header="infer",encoding='gbk')
        file_name=file.strip('-Raw.csv')
        data.columns = ["id", "Time", file_name]
        all_data=all_data.append(data[file_name])
    trans_data=all_data.T
    all_data=pd.concat([pd_time,trans_data],axis=1).drop(['Id','Value'],axis=1).set_index('Time')
    return all_data

In [3]:
#文件路径
x_April_path='/Mine/new/生产参数记录表（固定时间间隔）-2018年4月'
x_May_path='/Mine/new/生产参数记录表（固定时间间隔）-2018年5月'
#采用流量算法，需要纳入的特征，压力用于生成反应总压力：
files=[ 'Z230进口硫酸压力-Raw.csv','Z230进口液氨压力-Raw.csv','Z230进口液氨流量-Raw.csv','Z230进口洗涤液压力-Raw.csv','Z230进口洗涤液浓度-Raw.csv',
       '成品重量-Raw.csv', '返料重量-Raw.csv',
       'Z221进口液氨流量-Raw.csv','Z220造粒机电流-Raw.csv',
       'F101鼓风机电流-Raw.csv', 'E301干燥窑转速-Raw.csv','E301干燥窑电流-Raw.csv','E301干燥窑进口烟气温度-Raw.csv','F101热风炉炉膛温度-Raw.csv',
       'F101热风炉鼓风压力-Raw.csv','L101斗提机电流-Raw.csv',
       'S101A振网筛电流-Raw.csv','S101B振网筛电流-Raw.csv',
       'S102A破碎机电流1-Raw.csv','S102A破碎机电流2-Raw.csv','S102B破碎机电流1-Raw.csv','S102B破碎机电流2-Raw.csv']

In [4]:
#定义读取函数
all_data=pd.DataFrame()
all_April_data=read_data(x_April_path,all_data)

In [5]:
all_data=pd.DataFrame()
all_May_data=read_data(x_May_path,all_data)
all_x_data=pd.concat([all_April_data,all_May_data],axis=0)

In [6]:
all_x_data['返料比']=all_x_data['返料重量']/all_x_data['成品重量']
all_x_data['总压力']=all_x_data['Z230进口洗涤液压力']+all_x_data['Z230进口液氨压力']+all_x_data['Z230进口硫酸压力']
all_x_data['含氮量']=all_x_data['Z230进口液氨压力']/all_x_data['总压力']
all_x_data['含磷量']=all_x_data['Z230进口洗涤液压力']/all_x_data['总压力']
all_x_data['硫酸量']=all_x_data['Z230进口硫酸压力']/all_x_data['总压力']
all_x_data['中和度']=all_x_data['Z230进口洗涤液压力']/all_x_data['Z230进口液氨压力']

In [7]:
April_y_data=pd.read_csv('/home/data/2.产品检验报告/产品检验报告2018-4-1.csv')
May_y_data=pd.read_csv('/home/data/2.产品检验报告/产品检验报告2018-5-1-sample.csv')
y_data=pd.concat([April_y_data,May_y_data],axis=0,sort=True)
y_data=pd.concat([y_data, pd.DataFrame(columns=['0_time','1_time','2_time'])],sort=True)
y_data.reset_index(drop=True,inplace=True)

In [8]:
y_data.head()

,0_time,1_time,2_time,id,nitrogen_content,particle_size,phosphorus_content,product_batch,total_nutrient,water_content
0,NaN,NaN,NaN,1804-1,16.55,94.2,44.33,4.5 16:00 - 00:00,60.88,2.82
1,NaN,NaN,NaN,1804-2,16.79,95.3,44.72,4.6 00:00 - 04:00,61.51,2.70
2,NaN,NaN,NaN,1804-3,16.49,93.3,45.00,4.6 04:00 - 08:00,61.49,2.76
3,NaN,NaN,NaN,1804-4,16.64,94.8,44.53,4.6 08:00 - 12:00,61.17,3.14
4,NaN,NaN,NaN,1804-5,16.48,95.8,44.93,4.6 12:00 - 16:00,61.41,2.76


In [9]:
for i in range(0,len(y_data)):
    sp_str=y_data['product_batch'][i].split(' ')
    ssp_str_0=sp_str[1].split(':')
    ssp_str_1=sp_str[3].split(':')
    if int(ssp_str_0[0])>int(ssp_str_1[0]):
        ssp_str_1[0]=int(ssp_str_1[0])+23
    else:
        ssp_str_1[0]=int(ssp_str_1[0])-1
    y_data['0_time'][i]=str(2018.)+sp_str[0] + ' ' + ssp_str_0[0] + ':00'+':00'
    y_data['2_time'][i]=str(2018.)+sp_str[0]+' '+str(ssp_str_1[0])+':59'+':59' 

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.


In [10]:
y_data['0_time']=pd.to_datetime(y_data['0_time'],format='%Y.%m.%d %H:%M:%S')
y_data['2_time']=pd.to_datetime(y_data['2_time'],format='%Y.%m.%d %H:%M:%S')
y_data['1_time']=y_data['0_time']+(y_data['2_time']-y_data['0_time'])/2

In [11]:
#要不要保留是否在该时间内有停车的数据
#从1804-2开始预测，包括上一时间段的含量数据和6小时前的开始的时长数据
#例如采用4.5日16-24点的含量数据和18-22点的传感器数据预测6日0-4点间的含量数据
re_x=pd.DataFrame()
std_x=pd.DataFrame()
for j in range(1,len(y_data)):#
    start=y_data['0_time'][j]-pd.Timedelta("7 Hours")
    end=y_data['2_time'][j]-pd.Timedelta("7 Hours")
    resample=all_x_data[start:end]
    drop_index=resample[(resample['成品重量']<=5)|(resample['返料重量']<=50)].index
    clean_data=resample.drop(index=drop_index)
    sample=pd.DataFrame(clean_data[start:end].mean()).T
    std=pd.DataFrame(clean_data[start:end].std()).T
    sample['Time']=start+pd.Timedelta("7 Hours")
    std['Time']=start+pd.Timedelta("7 Hours")
    sample.index=[j-1]
    std.index=[j-1]
    re_x=re_x.append(sample)
    std_x=std_x.append(std)

In [12]:
std_x.head(10)

,Z230进口硫酸压力,Z230进口液氨压力,Z230进口液氨流量,Z230进口洗涤液压力,Z230进口洗涤液浓度,成品重量,返料重量,Z221进口液氨流量,Z220造粒机电流,F101鼓风机电流,...,S102A破碎机电流2,S102B破碎机电流1,S102B破碎机电流2,返料比,总压力,含氮量,含磷量,硫酸量,中和度,Time
0,53.460441,18.755498,0.097431,3.916976,0.001926,3.087333,4.737912,0.131733,1.456202,0.654665,...,0.702236,0.976882,1.511608,2.292930,68.207083,0.005673,0.019110,0.022075,0.060067,2018-04-06 00:00:00
1,41.486874,114.770271,0.977607,60.304386,0.079424,4.521714,8.875399,0.521876,3.596639,0.965164,...,0.423024,1.369409,2.090400,3.186693,90.219473,0.061994,0.045105,0.029816,0.179920,2018-04-06 04:00:00
2,48.057434,103.646926,0.695602,14.981667,0.006289,1.769457,2.762528,0.097829,2.553287,0.535665,...,0.163450,0.822879,1.302384,0.370525,121.221475,0.039917,0.030782,0.029218,0.189097,2018-04-06 08:00:00
3,43.161552,103.040595,0.750024,18.518589,0.008839,2.238121,3.183116,0.299672,2.725700,0.546179,...,0.373373,1.046950,1.399777,0.516486,143.757456,0.034034,0.027833,0.019724,0.184893,2018-04-06 12:00:00
4,40.881239,126.750660,1.019481,22.717718,0.004475,2.503858,4.358716,0.129390,3.349583,0.454074,...,0.664899,1.305852,1.890653,0.536009,152.491207,0.043014,0.026889,0.028115,0.182024,2018-04-06 16:00:00
5,54.728485,118.669849,0.900292,20.086910,0.007897,2.341727,3.820136,0.243935,2.959692,0.600326,...,0.489556,1.009547,1.224464,0.524499,160.226343,0.038877,0.030738,0.027005,0.195113,2018-04-06 20:00:00
6,40.783291,69.409288,0.583752,13.442486,0.002541,1.634545,2.980552,0.042218,2.786088,0.464011,...,0.465343,0.911837,1.282593,0.314388,80.565334,0.026907,0.018314,0.023556,0.115473,2018-04-07 00:00:00
7,51.243489,82.038841,0.640895,19.173782,0.004404,1.404002,2.802620,0.068769,2.438403,0.508631,...,0.254814,0.565508,0.842629,0.265483,101.795271,0.030038,0.024349,0.027995,0.137916,2018-04-07 04:00:00
8,45.844119,111.170932,0.854879,25.364481,0.004394,2.058371,3.323780,0.265157,2.740841,1.300066,...,0.153506,0.754358,0.965661,0.393220,138.730416,0.038991,0.028981,0.027716,0.200001,2018-04-07 08:00:00
9,42.137856,100.413752,0.706504,22.788666,0.004885,1.402587,3.509109,0.238043,2.276778,0.610381,...,0.528924,0.723146,0.943729,0.267069,123.484471,0.034701,0.031512,0.024198,0.192413,2018-04-07 12:00:00


In [13]:
#参数是否要改成144，需要用
add_data=y_data[['nitrogen_content','phosphorus_content','total_nutrient','water_content','particle_size']][0:143]
per_x_data=pd.concat([re_x,add_data],axis=1) 

In [14]:
all_use_columns=['Z230进口液氨流量','Z220造粒机电流',
                 'F101鼓风机电流','E301干燥窑电流','F101热风炉鼓风压力','S101A振网筛电流','S101B振网筛电流','E301干燥窑转速',
                 'S102A破碎机电流1','S102A破碎机电流2','S102B破碎机电流1','S102B破碎机电流2','Z230进口液氨压力', 'Z221进口液氨流量',
                 '返料比', '总压力', '含氮量', '含磷量', '中和度','硫酸量','E301干燥窑进口烟气温度','F101热风炉炉膛温度',
                 'nitrogen_content', 'phosphorus_content', 'total_nutrient','water_content','particle_size']
scaled_x=per_x_data[all_use_columns]/per_x_data[all_use_columns].mean()
print(scaled_x.shape)

(220, 27)


In [15]:
per_x_data[all_use_columns].tail(10)

,Z230进口液氨流量,Z220造粒机电流,F101鼓风机电流,E301干燥窑电流,F101热风炉鼓风压力,S101A振网筛电流,S101B振网筛电流,E301干燥窑转速,S102A破碎机电流1,S102A破碎机电流2,...,含磷量,中和度,硫酸量,E301干燥窑进口烟气温度,F101热风炉炉膛温度,nitrogen_content,phosphorus_content,total_nutrient,water_content,particle_size
210,4.859533,124.241427,34.793079,237.369968,-0.485745,1.128490,1.478012,2.95967,48.310041,40.997744,...,0.360748,1.133073,0.320827,400.676717,835.081429,NaN,NaN,NaN,NaN,NaN
211,4.993453,126.223834,34.611333,234.502926,-0.485226,1.105218,1.452097,2.95967,46.167959,39.683222,...,0.362834,1.114100,0.311416,400.084404,847.453920,NaN,NaN,NaN,NaN,NaN
212,4.964841,124.521142,34.196592,233.624322,-0.484419,1.103930,1.448589,2.95967,46.118278,39.498440,...,0.384952,1.252318,0.306984,397.610011,846.836976,NaN,NaN,NaN,NaN,NaN
213,4.903845,129.043779,34.846837,239.426295,-0.485455,1.106183,1.459564,2.95967,47.454114,40.415125,...,0.380285,1.226249,0.309351,400.962294,843.262261,NaN,NaN,NaN,NaN,NaN
214,4.906708,127.909364,33.733594,246.397601,-0.485288,1.104207,1.445770,2.95967,49.146745,41.215702,...,0.374980,1.205900,0.313854,389.958729,843.125838,NaN,NaN,NaN,NaN,NaN
215,4.872248,126.703251,34.077972,240.989872,-0.485520,1.100403,1.446886,2.95967,47.286135,40.179185,...,0.374268,1.266395,0.330104,389.123884,842.910680,NaN,NaN,NaN,NaN,NaN
216,4.164726,120.488054,34.166941,227.547850,-0.485153,1.103525,1.461167,2.95967,47.096384,39.931623,...,0.327752,1.093907,0.247256,386.454953,837.584045,NaN,NaN,NaN,NaN,NaN
217,4.825660,128.752530,32.817405,237.667354,-0.485330,1.127449,1.517598,2.95967,46.349371,40.137886,...,0.355781,1.284441,0.366726,403.952346,854.583300,NaN,NaN,NaN,NaN,NaN
218,4.801597,131.673810,34.308504,240.756509,-0.485344,1.120506,1.517465,2.95967,46.419305,40.476590,...,0.355891,1.169576,0.339707,388.475125,854.445595,NaN,NaN,NaN,NaN,NaN
219,4.801329,131.910375,34.744689,242.040508,-0.485392,1.110529,1.514384,2.95967,47.817084,41.514733,...,0.363541,1.184272,0.329445,385.011120,846.488475,NaN,NaN,NaN,NaN,NaN


In [16]:
per_x_data[all_use_columns].describe()

,Z230进口液氨流量,Z220造粒机电流,F101鼓风机电流,E301干燥窑电流,F101热风炉鼓风压力,S101A振网筛电流,S101B振网筛电流,E301干燥窑转速,S102A破碎机电流1,S102A破碎机电流2,...,含磷量,中和度,硫酸量,E301干燥窑进口烟气温度,F101热风炉炉膛温度,nitrogen_content,phosphorus_content,total_nutrient,water_content,particle_size
count,220.000000,220.000000,220.000000,220.000000,220.000000,220.000000,220.000000,220.000000,220.000000,220.000000,...,220.000000,220.000000,220.000000,220.000000,220.000000,143.000000,143.000000,143.000000,143.000000,143.000000
mean,4.614035,126.047832,34.360358,230.949160,-0.481801,1.346203,1.452341,2.958610,46.481734,41.844833,...,0.379959,1.233291,0.303771,387.106215,846.344619,16.314126,44.478671,60.792797,2.797692,92.532867
std,0.499325,9.016544,2.289812,16.834988,0.018631,0.177325,0.023660,0.006933,3.411305,8.106106,...,0.032003,0.152665,0.027948,8.409184,13.565310,0.242407,0.338081,0.355638,0.295284,3.656497
min,0.593213,2.396700,1.299902,1.055855,-0.486310,0.877364,1.412037,2.885543,5.866760,5.081440,...,0.021968,0.037434,0.109102,338.934870,707.626631,15.720000,43.620000,59.920000,1.950000,79.100000
25%,4.561686,124.917305,34.185929,228.856975,-0.485630,1.137465,1.435046,2.959670,44.949795,38.944467,...,0.367607,1.153289,0.296158,382.133287,840.365656,16.100000,44.270000,60.540000,2.595000,91.150000
50%,4.707933,126.672081,34.540912,231.884347,-0.485428,1.465335,1.445432,2.959670,46.501197,40.026821,...,0.380283,1.237330,0.304852,387.057874,848.468992,16.340000,44.510000,60.790000,2.790000,93.300000
75%,4.811942,128.848694,34.827205,235.426173,-0.485263,1.480709,1.465248,2.959670,48.317919,40.961592,...,0.394088,1.309906,0.314133,392.699401,854.153795,16.515000,44.680000,61.020000,2.955000,94.950000
max,5.020790,133.834210,35.731949,246.397601,-0.307198,1.669277,1.533158,2.960908,51.031313,70.452538,...,0.435591,1.611374,0.404870,408.475334,866.846942,16.790000,45.220000,61.760000,3.600000,98.600000


In [17]:
scaled_x.head(5)

,Z230进口液氨流量,Z220造粒机电流,F101鼓风机电流,E301干燥窑电流,F101热风炉鼓风压力,S101A振网筛电流,S101B振网筛电流,E301干燥窑转速,S102A破碎机电流1,S102A破碎机电流2,...,含磷量,中和度,硫酸量,E301干燥窑进口烟气温度,F101热风炉炉膛温度,nitrogen_content,phosphorus_content,total_nutrient,water_content,particle_size
0,1.075516,0.979627,1.039918,1.031470,1.007712,0.864363,1.005510,0.975304,1.048482,1.644608,...,0.981392,1.057609,1.123322,1.049483,1.015684,1.014458,0.996657,1.001434,1.007974,1.018017
1,1.046069,0.983211,0.990239,1.008755,0.935497,0.861727,0.993608,0.981357,1.017962,1.636352,...,0.958410,0.919706,1.005896,0.950201,0.986017,1.029169,1.005426,1.011797,0.965081,1.029904
2,1.021280,0.985404,1.021966,1.015855,1.007313,0.851401,0.983575,0.991669,1.037776,1.614171,...,1.119217,1.153583,0.893133,1.037873,1.024225,1.010780,1.011721,1.011469,0.986527,1.008290
3,1.011717,0.976035,1.009306,1.023461,1.007572,0.851327,0.983755,0.993755,1.059314,1.621062,...,1.056613,1.085547,0.969242,1.017389,1.019728,1.019975,1.001154,1.006205,1.122354,1.024501
4,0.958497,0.969866,1.023944,1.005927,1.007602,0.852537,0.985031,0.995084,1.049610,1.632378,...,0.982790,0.980716,1.027097,1.024318,1.007207,1.010168,1.010147,1.010153,0.986527,1.035308


In [18]:
scaled_x.describe()

,Z230进口液氨流量,Z220造粒机电流,F101鼓风机电流,E301干燥窑电流,F101热风炉鼓风压力,S101A振网筛电流,S101B振网筛电流,E301干燥窑转速,S102A破碎机电流1,S102A破碎机电流2,...,含磷量,中和度,硫酸量,E301干燥窑进口烟气温度,F101热风炉炉膛温度,nitrogen_content,phosphorus_content,total_nutrient,water_content,particle_size
count,220.000000,220.000000,220.000000,220.000000,220.000000,220.000000,220.000000,220.000000,220.000000,220.000000,...,220.000000,220.000000,220.000000,220.000000,220.000000,143.000000,143.000000,143.000000,143.000000,143.000000
mean,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
std,0.108219,0.071533,0.066641,0.072895,0.038669,0.131722,0.016291,0.002343,0.073390,0.193718,...,0.084227,0.123787,0.092002,0.021723,0.016028,0.014859,0.007601,0.005850,0.105545,0.039516
min,0.128567,0.019014,0.037831,0.004572,0.637603,0.651732,0.972249,0.975304,0.126216,0.121435,...,0.057816,0.030353,0.359157,0.875560,0.836098,0.963582,0.980695,0.985643,0.697003,0.854831
25%,0.988654,0.991031,0.994924,0.990941,1.007186,0.844943,0.988092,1.000358,0.967042,0.930688,...,0.967493,0.935131,0.974937,0.987154,0.992936,0.986875,0.995309,0.995842,0.927550,0.985055
50%,1.020351,1.004952,1.005255,1.004049,1.007529,1.088495,0.995243,1.000358,1.000419,0.956554,...,1.000854,1.003275,1.003558,0.999875,1.002510,1.001586,1.000704,0.999954,0.997250,1.008290
75%,1.042892,1.022221,1.013587,1.019385,1.007948,1.099915,1.008887,1.000358,1.039503,0.978892,...,1.037186,1.062123,1.034112,1.014449,1.009227,1.012313,1.004526,1.003737,1.056228,1.026122
max,1.088156,1.061773,1.039918,1.066891,1.009360,1.239990,1.055646,1.000777,1.097879,1.683662,...,1.146416,1.306564,1.332814,1.055202,1.024225,1.029169,1.016667,1.015910,1.286775,1.065567


In [19]:
x1_cols=[ 'Z221进口液氨流量','Z220造粒机电流', 'E301干燥窑电流', 'F101热风炉鼓风压力','返料比',
         '总压力', '含氮量', '含磷量', '中和度', '硫酸量','nitrogen_content','E301干燥窑进口烟气温度']
x2_cols=['Z221进口液氨流量','Z220造粒机电流','E301干燥窑电流', 'F101热风炉鼓风压力','返料比',
         '总压力', '含氮量', '含磷量', '中和度', '硫酸量','phosphorus_content','E301干燥窑进口烟气温度']
x3_cols=[ 'Z221进口液氨流量','Z220造粒机电流','E301干燥窑电流', 'F101热风炉鼓风压力','返料比',
         '总压力', '含氮量', '含磷量', '中和度', '硫酸量','total_nutrient','E301干燥窑进口烟气温度']
x4_cols=[ 'Z221进口液氨流量','Z220造粒机电流','E301干燥窑电流', 'F101热风炉鼓风压力','返料比',
         '总压力', '含氮量', '含磷量', '中和度', '硫酸量','water_content','E301干燥窑进口烟气温度']
x5_cols=['S102A破碎机电流1','返料比','总压力', '含氮量','含磷量','中和度','硫酸量','E301干燥窑进口烟气温度']

In [20]:
x1=scaled_x[x1_cols][0:143].values
x2=scaled_x[x2_cols][0:143].values
x3=scaled_x[x3_cols][0:143].values
x4=scaled_x[x4_cols][0:143].values
x5=scaled_x[x5_cols][37:143].values

In [21]:
per_y_data=y_data[['nitrogen_content','phosphorus_content','total_nutrient','water_content','particle_size']][1:144]
per_y_data.columns = ["y1", "y2", 'y3','y4','y5']
per_y_data.reset_index(drop=True,inplace=True)

In [22]:
y1=per_y_data['y1'][0:143].values
print(y1)
y2=per_y_data['y2'][0:143].values
y3=per_y_data['y3'][0:143].values
y4=per_y_data['y4'][0:143].values
y5=per_y_data['y5'][37:143].values

[ 16.79  16.49  16.64  16.48  16.54  16.56  16.54  16.56  16.49  16.4
  16.69  16.53  16.36  16.38  16.62  16.53  16.37  16.47  16.27  16.3
  16.24  16.33  16.43  16.76  16.5   16.45  16.6   16.6   16.44  16.51
  16.61  16.54  16.57  16.5   16.25  16.48  16.66  16.39  16.46  16.38
  16.2   16.18  16.53  16.62  16.52  16.52  16.6   16.53  16.69  16.25
  16.65  16.71  16.64  16.56  16.44  16.29  16.52  16.44  16.37  16.3
  16.53  16.53  16.43  16.46  16.31  16.43  16.35  16.3   16.4   16.43
  16.34  16.45  16.77  16.78  16.72  16.27  16.44  16.52  16.25  16.21
  16.2   16.21  16.28  16.32  16.25  16.26  16.42  16.19  16.49  16.2
  16.17  16.18  16.35  16.09  16.12  16.07  16.08  16.09  16.08  15.96
  16.16  16.23  16.09  16.11  16.14  16.24  15.88  16.01  16.    15.97
  15.92  16.02  16.    15.99  16.02  15.99  15.99  16.44  16.04  16.3
  16.23  16.08  16.23  16.18  15.96  15.99  16.03  16.35  16.04  16.05
  16.04  16.09  15.96  16.07  15.88  15.77  15.72  15.89  15.87  15.86
  15.96  16

# 开始训练模型

In [23]:
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
import numpy as np
from sklearn.model_selection import KFold
class stacking(BaseEstimator, RegressorMixin, TransformerMixin):
    def __init__(self,mod,meta_model):
        self.mod = mod
        self.meta_model = meta_model
        self.kf = KFold(n_splits=5, random_state=42, shuffle=True)
        
    def fit(self,X,y):
        self.saved_model = [list() for i in self.mod]
        oof_train = np.zeros((X.shape[0], len(self.mod)))
        
        for i,model in enumerate(self.mod):
            for train_index, val_index in self.kf.split(X,y):
                renew_model = clone(model)
                renew_model.fit(X[train_index], y[train_index])
                self.saved_model[i].append(renew_model)
                oof_train[val_index,i] = renew_model.predict(X[val_index])
        
        self.meta_model.fit(oof_train,y)
        return self
    
    def predict(self,X):
        whole_test = np.column_stack([np.column_stack(model.predict(X) for model in single_model).mean(axis=1) 
                                      for single_model in self.saved_model]) 
        return self.meta_model.predict(whole_test)
    
    def get_oof(self,X,y,test_X):
        oof = np.zeros((X.shape[0],len(self.mod)))
        test_single = np.zeros((test_X.shape[0],5))
        test_mean = np.zeros((test_X.shape[0],len(self.mod)))
        for i,model in enumerate(self.mod):
            for j, (train_index,val_index) in enumerate(self.kf.split(X,y)):
                clone_model = clone(model)
                clone_model.fit(X[train_index],y[train_index])
                oof[val_index,i] = clone_model.predict(X[val_index])
                test_single[:,j] = clone_model.predict(test_X)
            test_mean[:,i] = test_single.mean(axis=1)
        return oof, test_mean

In [24]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet
from sklearn.ensemble import GradientBoostingRegressor
from xgboost import XGBRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.svm import SVR

In [25]:
knr_1=KernelRidge()
ela_1=ElasticNet(alpha=5e-05,l1_ratio=0.8,max_iter=10000)
gbr_1= GradientBoostingRegressor(learning_rate=0.1, max_depth=1, n_estimators=80)
xg_1=XGBRegressor(learning_rate=0.2, max_depth=1, n_estimators=40)
svr_1=SVR(C=100, gamma=0.1, kernel='rbf')
stack_model_1 = stacking(mod=[ela_1,gbr_1,xg_1,svr_1],meta_model=knr_1)
stack_model_1.fit(x1,y1)
for m in range(142,220):
    if m<143:   
        test_x1=scaled_x[x1_cols][m:m+1].values
        test_y=stack_model_1.predict(test_x1).tolist()
        scaled_x.loc[m+1,'nitrogen_content']=round(test_y[0],2)
    else:    
        test_x1=scaled_x[x1_cols][m:m+1].values
        test_x1[0][10]=test_x1[0][10]/16.3
        test_y=stack_model_1.predict(test_x1).tolist()
        scaled_x.loc[m+1,'nitrogen_content']=test_y[0]

16.3
16.2479453944
16.1995197563
16.1614443829
16.149554799
16.1084936968
16.0481217527
16.0686728917
16.0644666599
16.1954079914
16.275932196
16.0952958019
16.1519814126
16.1957504366
16.1845100429
16.2354898495
16.149984144
16.1956571187
16.2106243836
16.1608666165
16.1100180644
16.1210790039
16.1702706647
16.1581271682
16.027128642
16.0732760987
16.0898150633
16.1163866757
16.1403724366
16.1061922696
16.0510463233
16.1671448655
16.1077163973
16.2011304665
16.1658126268
16.1891857312
16.133934899
16.1145590555
16.1092663084
16.0754925756
16.1131664714
16.1516477447
16.1439594231
16.1297071646
16.116681847
16.1508968291
16.118248265
16.1207093268
16.1128180798
16.1669120796
16.1708403638
16.169889647
16.2719868922
16.3160065566
16.2913917651
16.2795970291
16.2668104832
16.3155845985
16.3117315391
16.2706648588
16.2923646855
16.307298841
16.2755780874
16.2380056455
16.2356792963
16.2428497387
16.2111443368
16.1585092018
16.1370148376
16.0993854518
16.04081364
16.1089079948
16.097854752

In [26]:
knr_2=KernelRidge()
ela_2=ElasticNet(alpha= 5e-05, l1_ratio= 0.8, max_iter= 10000)
gbr_2= GradientBoostingRegressor(learning_rate= 0.05, max_depth= 4, n_estimators= 40)
xg_2=XGBRegressor(learning_rate= 0.1, max_depth= 1, n_estimators= 80)
svr_2=SVR(C= 10, gamma= 0.5, kernel= 'rbf')
stack_model_2 = stacking(mod=[ela_2,gbr_2,xg_2,svr_2],meta_model=knr_2)
stack_model_2.fit(x2,y2)
for m in range(142,220):
    if m<143:   
        test_x2=scaled_x[x2_cols][m:m+1].values
        test_y=stack_model_2.predict(test_x2).tolist()
        scaled_x.loc[m+1,'phosphorus_content']=round(test_y[0],2)
    else:    
        test_x2=scaled_x[x2_cols][m:m+1].values
        test_x2[0][10]=test_x2[0][10]/44.47
        test_y=stack_model_2.predict(test_x2).tolist()
        scaled_x.loc[m+1,'phosphorus_content']=test_y[0]

In [27]:
knr_3=KernelRidge()
ela_3=ElasticNet(alpha= 5e-05, l1_ratio= 0.8, max_iter= 10000)
gbr_3= GradientBoostingRegressor(learning_rate= 0.02, max_depth= 3, n_estimators= 100)
xg_3=XGBRegressor(learning_rate= 0.1, max_depth= 1, n_estimators= 80)
svr_3=SVR(C= 10, gamma= 0.1, kernel= 'rbf')
stack_model_3 = stacking(mod=[ela_3,gbr_3,xg_3,svr_3],meta_model=knr_3)
stack_model_3.fit(x3,y3)
for m in range(142,220):
    if m<143:   
        test_x3=scaled_x[x3_cols][m:m+1].values
        test_y=stack_model_3.predict(test_x3).tolist()
        scaled_x.loc[m+1,'total_nutrient']=round(test_y[0],2)
    else:    
        test_x3=scaled_x[x3_cols][m:m+1].values
        test_x3[0][10]=test_x3[0][10]/60.792
        test_y=stack_model_3.predict(test_x3).tolist()
        scaled_x.loc[m+1,'total_nutrient']=test_y[0]

In [28]:
knr_4=KernelRidge()
ela_4=ElasticNet(alpha= 5e-05, l1_ratio= 0.8, max_iter= 10000)
gbr_4= GradientBoostingRegressor(learning_rate= 0.1, max_depth= 1, n_estimators= 60)
xg_4=XGBRegressor(learning_rate= 0.05, max_depth= 1, n_estimators= 200)
svr_4=SVR(C= 1, gamma= 0.5, kernel= 'rbf')
stack_model_4 = stacking(mod=[ela_4,gbr_4,xg_4,svr_4],meta_model=knr_4)
stack_model_4.fit(x4,y4)
for m in range(142,220):
    if m<143:   
        test_x4=scaled_x[x4_cols][m:m+1].values
        test_y=stack_model_4.predict(test_x4).tolist()
        scaled_x.loc[m+1,'water_content']=round(test_y[0],2)
    else:    
        test_x4=scaled_x[x4_cols][m:m+1].values
        test_x4[0][10]=test_x4[0][10]/2.798
        test_y=stack_model_4.predict(test_x4).tolist()
        scaled_x.loc[m+1,'water_content']=test_y[0]

In [29]:
knr_5=KernelRidge()
ela_5=ElasticNet(alpha= 0.0005, l1_ratio= 0.8, max_iter= 10000)
gbr_5= GradientBoostingRegressor(learning_rate= 0.1, max_depth= 2, n_estimators= 20)
xg_5=XGBRegressor(learning_rate= 0.05, max_depth= 1, n_estimators= 200)
svr_5=SVR(C= 100, gamma= 1, kernel= 'rbf')
stack_model_5 = stacking(mod=[ela_5,gbr_5,xg_5,svr_5],meta_model=knr_5)
stack_model_5.fit(x5,y5)
for m in range(142,220): 
    test_x5=scaled_x[x5_cols][m:m+1].values
    test_y=stack_model_5.predict(test_x5).tolist()
    scaled_x.loc[m+1,'particle_size']=test_y[0]

In [30]:
scaled_x.tail(10)

,Z230进口液氨流量,Z220造粒机电流,F101鼓风机电流,E301干燥窑电流,F101热风炉鼓风压力,S101A振网筛电流,S101B振网筛电流,E301干燥窑转速,S102A破碎机电流1,S102A破碎机电流2,...,含磷量,中和度,硫酸量,E301干燥窑进口烟气温度,F101热风炉炉膛温度,nitrogen_content,phosphorus_content,total_nutrient,water_content,particle_size
211,1.082231,1.001396,1.007304,1.015388,1.007110,0.820989,0.999832,1.000358,0.993249,0.948342,...,0.954929,0.903356,1.025167,1.033526,1.001311,16.137015,44.774826,60.930513,2.924742,93.748553
212,1.076030,0.987888,0.995234,1.011583,1.005433,0.820033,0.997417,1.000358,0.992181,0.943926,...,1.013142,1.015427,1.010576,1.027134,1.000582,16.099385,44.720358,60.833938,2.945424,94.671660
213,1.062811,1.023768,1.014158,1.036706,1.007584,0.821706,1.004974,1.000358,1.020920,0.965833,...,1.000860,0.994290,1.018370,1.035794,0.996358,16.040814,44.694888,60.803583,2.959571,94.609166
214,1.063431,1.014768,0.981759,1.066891,1.007238,0.820239,0.995476,1.000358,1.057335,0.984965,...,0.986897,0.977790,1.033192,1.007369,0.996197,16.108908,44.689896,60.861540,2.889340,94.005621
215,1.055963,1.005200,0.991782,1.043476,1.007718,0.817413,0.996244,1.000358,1.017306,0.960195,...,0.985023,1.026842,1.086686,1.005212,0.995943,16.097855,44.751974,60.948126,2.870679,91.104523
216,0.902621,0.955892,0.994371,0.985272,1.006958,0.819732,1.006077,1.000358,1.013223,0.954278,...,0.862600,0.886982,0.813956,0.998318,0.989649,16.071854,44.741433,60.890645,2.908080,93.723423
217,1.045865,1.021458,0.955095,1.029089,1.007326,0.837503,1.044932,1.000358,0.997152,0.959208,...,0.936368,1.041474,1.207243,1.043518,1.009734,15.777339,44.702253,60.481133,3.211510,93.859882
218,1.040650,1.044634,0.998491,1.042465,1.007354,0.832346,1.044841,1.000358,0.998657,0.967302,...,0.936656,0.948337,1.118299,1.003536,1.009572,16.012668,44.631023,60.694496,3.025360,94.887174
219,1.040592,1.046510,1.011185,1.048025,1.007454,0.824934,1.042719,1.000358,1.028728,0.992111,...,0.956790,0.960253,1.084516,0.994588,1.000170,16.052290,44.567909,60.618208,3.057427,94.215053
220,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,16.065423,44.625223,60.711032,2.981077,93.441595


In [31]:
#将所预测的内容输出到制定文件中
final_columns=['nitrogen_content','phosphorus_content','total_nutrient','water_content','particle_size']
result=scaled_x[final_columns][143:220]

     nitrogen_content  phosphorus_content  total_nutrient  water_content  \
143         16.300000           44.790000       61.010000       2.890000   
144         16.247945           44.717900       60.867089       2.737073   
145         16.199520           44.629882       60.769707       2.870331   
146         16.161444           44.641028       60.718181       2.904836   
147         16.149555           44.796366       60.893664       2.795366   
148         16.108494           44.676015       60.819622       2.831589   
149         16.048122           44.632540       60.705147       2.897510   
150         16.068673           44.525980       60.579743       2.939563   
151         16.064467           44.605528       60.710727       2.866815   
152         16.195408           44.690681       60.873872       2.749018   
153         16.275932           44.693287       60.950795       2.707864   
154         16.095296           44.642831       60.691725       2.897388   
155         

In [32]:
id=y_data['id'][144:221].values
print(id)

['1805-1' '1805-2' '1805-3' '1805-4' '1805-5' '1805-6' '1805-7' '1805-8'
 '1805-9' '1805-10' '1805-11' '1805-12' '1805-13' '1805-14' '1805-15'
 '1805-16' '1805-17' '1805-18' '1805-19' '1805-20' '1805-21' '1805-22'
 '1805-23' '1805-24' '1805-25' '1805-26' '1805-27' '1805-28' '1805-29'
 '1805-30' '1805-31' '1805-32' '1805-33' '1805-34' '1805-35' '1805-36'
 '1805-37' '1805-38' '1805-39' '1805-40' '1805-41' '1805-42' '1805-43'
 '1805-44' '1805-45' '1805-46' '1805-47' '1805-48' '1805-49' '1805-50'
 '1805-51' '1805-52' '1805-53' '1805-54' '1805-55' '1805-56' '1805-57'
 '1805-58' '1805-59' '1805-60' '1805-61' '1805-62' '1805-63' '1805-64'
 '1805-65' '1805-66' '1805-67' '1805-68' '1805-69' '1805-70' '1805-71'
 '1805-72' '1805-73' '1805-74' '1805-75' '1805-76' '1805-77']


In [33]:
output=pd.DataFrame({"id":id,"phosphorus_content":result['phosphorus_content'].values,"nitrogen_content":result['nitrogen_content'].values
                  ,'total_nutrient':result['total_nutrient'].values,'water_content':result['water_content'].values,
                  'particle_size':result['particle_size'].values})

In [34]:
output.head(10)

,id,phosphorus_content,nitrogen_content,total_nutrient,water_content,particle_size
0,1805-1,44.790000,16.300000,61.010000,2.890000,94.414473
1,1805-2,44.717900,16.247945,60.867089,2.737073,94.726082
2,1805-3,44.629882,16.199520,60.769707,2.870331,95.507826
3,1805-4,44.641028,16.161444,60.718181,2.904836,95.710375
4,1805-5,44.796366,16.149555,60.893664,2.795366,93.957743
5,1805-6,44.676015,16.108494,60.819622,2.831589,93.739234
6,1805-7,44.632540,16.048122,60.705147,2.897510,94.703487
7,1805-8,44.525980,16.068673,60.579743,2.939563,94.337057
8,1805-9,44.605528,16.064467,60.710727,2.866815,94.476103
9,1805-10,44.690681,16.195408,60.873872,2.749018,94.091976


In [35]:
output.tail(10)

,id,phosphorus_content,nitrogen_content,total_nutrient,water_content,particle_size
67,1805-68,44.650610,16.158509,60.831855,2.950174,94.472900
68,1805-69,44.774826,16.137015,60.930513,2.924742,93.748553
69,1805-70,44.720358,16.099385,60.833938,2.945424,94.671660
70,1805-71,44.694888,16.040814,60.803583,2.959571,94.609166
71,1805-72,44.689896,16.108908,60.861540,2.889340,94.005621
72,1805-73,44.751974,16.097855,60.948126,2.870679,91.104523
73,1805-74,44.741433,16.071854,60.890645,2.908080,93.723423
74,1805-75,44.702253,15.777339,60.481133,3.211510,93.859882
75,1805-76,44.631023,16.012668,60.694496,3.025360,94.887174
76,1805-77,44.567909,16.052290,60.618208,3.057427,94.215053


In [36]:
output.set_index('id',inplace=True)

In [38]:
#output.to_csv('/home/code/result.csv',encoding='UTF-8')